# Исследование надёжности заёмщиков

## Описание проекта

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. 

В нашем распоряжении входные данные от банка - статистика платежеспособности клиентов.

Исходя из поставленной задачи разделим исследование на несколько частей:

#### Часть 1. Изучение общей информации
* [1.1. Загрузка библиотек, импорт файла с данными](#1.1.)
* [1.2. Общая информация о данных](#1.2.)

#### Часть 2. Предобработка данных 
* [2.1. Обработка пропущенных значений](#2.1.)
* [2.2. Замена типа данных](#2.2.)
* [2.3. Обработка дубликатов](#2.3.)
* [2.4. Лемматизация](#2.4.)
* [2.5. Категоризация данных](#2.5.)

#### Часть 3. Исследование надёжности заёмщиков
* [3.1. Зависимость между наличием детей и возвратом кредита в срок](#3.1.)
* [3.2. Зависимость между семейным положением и возвратом кредита в срок](#3.2.)
* [3.3. Зависимость между уровнем дохода и возвратом кредита в срок](#3.3.)
* [3.4. Влияние целей кредита на его возврат в срок](#3.4.)

#### Общий вывод

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Часть 1. Изучение общей информации

#### 1.1. Загрузка библиотек, импорт файла с данными <a id='1.1.'></a> 

Загрузим библиотеки, необходимые для проведения исследования.

In [1]:
# Импортируем необходимые библиотеки
import pandas as pd
from pymystem3 import Mystem
from collections import Counter

In [2]:
# Прочитаем данные методом read_csv и посмотрим на исхдный датафейм
df = pd.read_csv('/datasets/data.csv')

# Выведем на экран 5 случайных строк датафрейма 
df.sample(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
8379,0,-4832.970082,33,Среднее,1,женат / замужем,0,F,сотрудник,0,175759.420057,покупка жилой недвижимости
13119,0,-2612.422848,38,среднее,1,женат / замужем,0,M,сотрудник,0,276711.431463,на покупку автомобиля
2653,0,-627.669772,23,среднее,1,гражданский брак,1,F,сотрудник,0,170199.047656,сыграть свадьбу
1321,0,-2137.382749,25,среднее,1,женат / замужем,0,M,компаньон,0,143189.018064,строительство собственной недвижимости
4485,0,331096.440833,52,среднее,1,женат / замужем,0,F,пенсионер,0,124401.325630,операции с жильем


#### 1.2. Общая информация о данных <a id='1.2.'></a> 

Посмотрим общую информацию о датафрейме и всем признакам и их типам, воспользовавшись методом info().

In [3]:
# Посмотрим общую информацию о датафрейме методом info()
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Разберем более подробно, какие в таблице столбцы и какую информацию они содержат:

* children — количество детей в семье  
* days_employed — общий трудовой стаж в днях  
* dob_years — возраст клиента в годах  
* education — уровень образования клиента  
* education_id — идентификатор уровня образования  
* family_status — семейное положение  
* family_status_id — идентификатор семейного положения  
* gender — пол клиента  
* income_type — тип занятости;  
* debt — имел ли задолженность по возврату кредитов  
* total_income — ежемесячный доход  
* purpose — цель получения кредита  

Количество значений в столбцах различается. Это говорит о том, что в данных есть пропущенные значения.

Более детально изучим статистические характеристики данных по каждому числовому признаку при помощи метода describe().

In [4]:
# Посмотрим на основные статистические характеристики данных
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


### Вывод

Каждая строка таблицы содержит персональные данные клиентов банка и информацию о цели кредита и имеющихся задолженностях.

Отметим несколько проблем, которы требуют решения:  
* пропуски в исходных данных  
* информация об образовании занесена словами, имеющими разный регистр букв
* часть значений, характеризующих количество детей, - отрицательные или равны 20

Также, в исходных данных присутствуют клиенты, возраст которых равен нулю. Скорее всего такие значения указаны намеренно с целью скрыть реальный возраст. Оставим такие данные без изменений.

### Часть 2. Предобработка данных

Предобработаем данные: исключим или найдем логичную замену пропущенным значениям, изменим типы данных там, где это будет необходимо, проверим данные на наличие дуликатов, выделим леммы в значениях столбца с целями получения кредита и категоризуем данные.

#### 2.1. Обработка пропущенных значений <a id='2.1.'></a> 

In [5]:
# Проверим данные на наличие дубликатов
df.isna().mean() 

children            0.000000
days_employed       0.100999
dob_years           0.000000
education           0.000000
education_id        0.000000
family_status       0.000000
family_status_id    0.000000
gender              0.000000
income_type         0.000000
debt                0.000000
total_income        0.100999
purpose             0.000000
dtype: float64

Видна прямая зависимость пропусков в столбцах с трудовым стажем и ежемесячным доходом. Основной причиной этому может быть нежелание клиентов предоставлять данную информацию для обработки. В данном случае нецелесообразно удалять данные строки, т.к. на них приходится до 10% данных, в виду чего, произведем замену.  

Начнем с замены пропусков в столбце со стажем на нулевые значения.

In [6]:
# Заменим пропуенные значения
df['days_employed'] = df['days_employed'].fillna('0') 

# Выведем на экран результат замены
df.sample(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
14398,1,-2199.68,28,высшее,0,гражданский брак,1,F,компаньон,0,131807.841945,свадьба
15402,0,-1708.31,36,среднее,1,в разводе,3,M,сотрудник,0,89025.854930,жилье
13454,0,-1850.45,46,среднее,1,женат / замужем,0,F,сотрудник,0,78670.989489,автомобиль
7393,0,-4990.59,47,среднее,1,женат / замужем,0,F,сотрудник,0,177458.037361,покупка недвижимости
8944,0,-841.759,23,высшее,0,женат / замужем,0,F,компаньон,0,379568.013616,автомобили


Посмотрим на строки с пропусками в ежемесячном доходе более детально, возможно, удастся проследить какие-либо закономерности, которые помогут заполнить недостающие значения.

In [7]:
# Рассмотрим детально строки с пропусками в ежемесячном доходе
df[df['total_income'].isnull()].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,0,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,0,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,0,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,0,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,0,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


Заполним пропуски в данных о ежемесячном дохоже медианными значениями по роду деятельности клиентов банка.

In [8]:
# Найдем уникальные значения типов занятости
print(df['income_type'].unique())

['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']


In [9]:
# Заменим пропущенные значения на медиану при помощи цикла 
income_type_employee = df['income_type'].unique()
for i in income_type_employee:
    median = df[(df['income_type'] == i) & (df['total_income']).notna()]['total_income'].median()
    df.loc[(df['income_type'] == i) & (df['total_income']).isna(),'total_income'] = median

# Проверим результат замены 
df.loc[12].reset_index()

,index,12
0,children,0
1,days_employed,0
2,dob_years,65
3,education,среднее
4,education_id,1
5,family_status,гражданский брак
6,family_status_id,1
7,gender,M
8,income_type,пенсионер
9,debt,0


In [10]:
# Проверим количество пропусков после замены
df.isna().sum() 

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

#### 2.2. Замена типа данных <a id='2.2.'></a> 

Приведем значения параметров - общий трудовой стаж в днях и ежемесячный доход к целому числу, т.к. дробные части дней и копейки в уровнях дохода нас не интересуют. Это значительно облегчит дальнейшую работу с этими данными. 

In [11]:
# Приведем данные к целочисленному типу
df['days_employed'] = df['days_employed'].astype(int) 
df['total_income'] = df['total_income'].astype(int)

In [12]:
# Проверим результаты замены
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


#### 2.3. Обработка дубликатов <a id='2.3.'></a> 

В данных со стратистикой о платежеспособности клиентов были найдены значения, принимающие разные регистры. Приведем показатели с информацией об образовании к единому регистру и проверим данные на наличие дубликатов.

In [13]:
# Найдем частоту значений 
df['education'].value_counts().reset_index()

,index,education
0,среднее,13750
1,высшее,4718
2,СРЕДНЕЕ,772
3,Среднее,711
4,неоконченное высшее,668
5,ВЫСШЕЕ,274
6,Высшее,268
7,начальное,250
8,Неоконченное высшее,47
9,НЕОКОНЧЕННОЕ ВЫСШЕЕ,29


In [14]:
# Приведем значения к нижнему регистру
df['education'] = df['education'].str.lower() 

# Проверим данные на наличие дубликатов
print('Количество дубликатов в данных:', df.duplicated().sum())

Количество дубликатов в данных: 71


Обнаруженные в данных дубликаты - удалим, т.к. в дальнейшем они смогут оказать отрицательное влияние на принятие решений в рамках данного исследования. 

In [15]:
# Избавимся от дубликатов
df = df.drop_duplicates().reset_index(drop=True) 

# Проверим данные на дубли после преобразований
print('Количество дубликатов в данных после преобразований:', df.duplicated().sum())

Количество дубликатов в данных после преобразований: 0


#### 2.4. Лемматизация <a id='2.4.'></a> 

Преобразуем значения столбца с целями получения кредита к их базовой форме - лемматизируем каждое значение путем написания функции. Полученные результаты сохраним в новый столбец таблицы и посчитаем количество употребления в нем каждой леммы, сравнив полученное количество результатов с исходным. 

In [16]:
# Напишем функцию для лемматизации
m = Mystem()

def lemmas(purpose):
    lemmas = ' '.join(m.lemmatize(purpose))
    if 'свадьба' in lemmas:
        return 'свадьба'
    if 'недвижимость' in lemmas:
        return 'недвижимость'
    if 'образование' in lemmas:
        return 'образование'
    if 'автомобиль' in lemmas:
        return 'автомобиль'
    if 'жилье' in lemmas:
        return 'недвижимость'
    return lemmas

# Сохраним полученные значения в новой столбце lemmas_purpose 
df['lemmas_purpose'] = df['purpose'].apply(lemmas) 

# Посчитаем количество упоминаний в тексе
print(Counter(df['lemmas_purpose']))

Counter({'недвижимость': 10811, 'автомобиль': 4306, 'образование': 4013, 'свадьба': 2324})


In [17]:
# Выведем 5 случайных строк обновленной таблицы
df.sample(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas_purpose
19077,2,-794,39,высшее,0,женат / замужем,0,M,сотрудник,1,129193,сделка с подержанным автомобилем,автомобиль
3095,0,369118,55,среднее,1,женат / замужем,0,M,пенсионер,0,75866,покупка недвижимости,недвижимость
12304,3,-2967,41,среднее,1,женат / замужем,0,F,сотрудник,1,172969,покупка жилья,недвижимость
3371,1,-7135,38,среднее,1,женат / замужем,0,F,сотрудник,0,170334,высшее образование,образование
3659,1,0,39,среднее,1,гражданский брак,1,F,сотрудник,0,142594,свадьба,свадьба


#### 2.5. Категоризация данных <a id='2.5.'></a> 

Данные столбца с информацией о возрасте клиента классифицируем на несколько групп по десятилетиям. Особое внимание уделим группе со значением возраста равным - 0. Присвоем ему - no_info, т.к. мы не обладаем никакой информацией о возрасте данных клиентов. Также, показатели дохода клиентов банка разделим на несколько категорий, что значительно облегчит нам проведение дальнейшего анализа и выявление связи данного показателя с выплатами по кредитам.

In [18]:
# Сегментируем данные с возрастом клиента
df['dob_years_id'] = pd.cut(df['dob_years'],[0, 18, 25, 35, 45, 55, 100]) 

In [43]:
# Построим сводную таблицу для наглядности
df.pivot_table(index='dob_years_id', values='dob_years',aggfunc=['count']).reset_index() 

,dob_years_id,count
,,dob_years
0,18-25,1232
1,26-35,5351
2,36-45,5620
3,46-55,4814
4,55+,3853
5,no_info,101


In [32]:
# Напишем функцию сегментации данных
def age_group(age):
    
    if age <= 0:
        return 'no_info'
    if age <= 25:
        return '18-25'
    if age <= 35:
        return '26-35'
    if age <= 45:
        return '36-45'
    if age <= 55:
        return '46-55'
    if age > 56:
        return '55+'
    
df['dob_years_id'] = df['dob_years'].apply(age_group)

In [33]:
# Зададим несколько диапозонов значений столбцу "ежемесячный доход"
df['total_income_grouped'] = pd.qcut(df['total_income'],5) 

### Вывод

На этапе предобработки в данных обнаружились не только пропуски и проблемы с регистром значений одного из столбцов, но дубликаты. Их удаление позволит провести наше исследование точнее.  
Поскольку сведения о рабочем стаже и уровне дохода важно сохранить для анализа, мы не удалим все пропущенные значения, а выполним логичную замену.   
Теперь все наши данные корректны и могут быть использованы в дальнейшей работе.

### Часть 3. Исследование надежности заемщиков

Дадим ответы на ряд вопросов о завимости между наличием детей, семейным положением, уровнем дохода и целью кредита на его возврат в срок.

#### 3.1. Зависимость между наличием детей и возвратом кредита в срок  <a id='3.1.'></a> 

In [36]:
# Узнаем долю задолженностей
print('Общая доля задолженности:', round(df['debt'].mean(), 2))

Общая доля задолженности: 0.08


In [41]:
# Узнаем количественное и процентное соотношение задолженности исходя из количества детей
df.pivot_table(index='children', values='debt',aggfunc=['mean', 'count']).reset_index()

,children,mean,count
,,debt,debt
0,0,0.075438,14091
1,1,0.091658,4855
2,2,0.094925,2128
3,3,0.081818,330
4,4,0.097561,41
5,5,0.000000,9


In [39]:
# Заменим отрицательное количество детей на 1
df.loc[df['children'] < 0, 'children'] = 1 

# Заменим значение, равно количеству детей - 20
df.loc[df['children'] == 20, 'children'] = 2 

In [42]:
# Построим сводную таблицу после устранения аномалий
df.pivot_table(index='children', values='debt',aggfunc=['mean', 'count']).reset_index()

,children,mean,count
,,debt,debt
0,0,0.075438,14091
1,1,0.091658,4855
2,2,0.094925,2128
3,3,0.081818,330
4,4,0.097561,41
5,5,0.000000,9


Общая доля должников банка составляет - 8%. Наиболее высокий процент задолженностей по оплате кредитов наблюдается в группе клиентов с количеством детей - 4, на их долю приходится почти - 10%. Наименьшее количество просроченных платежей у клиентов без детей - примерно 7,5%. Полностью отсутствуют долги по кредитам у клиентов с количеством детей - 5.
Обнаруженные аномальные значения, такие как -1 и 20 были заменены на значения, равные количеству детей 1 и 2. Избавление от данных ошибочных показателей позволило провести более качественный анализ данных.  

#### 3.2. Зависимость между семейным положением и возвратом кредита в срок  <a id='3.2.'></a> 

In [44]:
# Узнаем количество клиентов по каждой категории
df['family_status'].value_counts().reset_index() 

,index,family_status
0,женат / замужем,12339
1,гражданский брак,4151
2,Не женат / не замужем,2810
3,в разводе,1195
4,вдовец / вдова,959


In [46]:
# Построим сводную таблицу
df.pivot_table(index='family_status_id', values='debt',aggfunc=['mean', 'count']).reset_index() 

,family_status_id,mean,count
,,debt,debt
0,0,0.075452,12339
1,1,0.093471,4151
2,2,0.065693,959
3,3,0.071130,1195
4,4,0.097509,2810


На категорию клиентов - вдовец/вдова приходится минимальное количество просрочек по выплатам кредитов - 6,5%, что ниже среднего показателя по задолженностям. Клиенты со статусом - не женат/не замужем чаще остальных не возвращают кредиты в срок. Таких клиентов - около 10%.  
Иными словами, наиболее надежные заемщики - это те, кто находится или когда-либо был в браке.

#### 3.3. Зависимость между уровнем дохода и возвратом кредита в срок <a id='3.3.'></a> 

In [49]:
# Зададим диапозон значений для большей наглядности
df['total_income_grouped'] = pd.qcut(df['total_income'],5) 

# Создадим сводную таблицу с зависимостью прострочки платежей от ежемесячного дохода 
df.groupby('total_income_grouped')['debt'].agg(['mean','count']).reset_index()

,total_income_grouped,mean,count
0,"(20666.999, 98537.6]",0.080168,4291
1,"(98537.6, 132134.4]",0.084130,4291
2,"(132134.4, 161335.0]",0.087413,4290
3,"(161335.0, 214618.2]",0.084130,4291
4,"(214618.2, 2265604.0]",0.069914,4291


Сгруппировав данные о ежемесячном доходе клиентов банка и рассмотрев зависимость задолженностей по выплатам от этого показателя, можно сделать вывод, что меньшее их количество наблюдается у клиентов с наибольшим доходом - 7%, что ниже среднего показателями по задолженостям.

#### 3.4. Влияние целей кредита на его возврат в срок <a id='3.4.'></a> 

In [50]:
# Построим сводную таблицу
df.pivot_table(index='lemmas_purpose', values='debt',aggfunc=['mean', 'count']).reset_index() 

,lemmas_purpose,mean,count
,,debt,debt
0,автомобиль,0.093590,4306
1,недвижимость,0.072334,10811
2,образование,0.092200,4013
3,свадьба,0.080034,2324


Клиенты банка, приследующие цели, связанные с недвижимостью, в меньшей степени допускают просрочку выплат. На них приходится примерно 7%, что ниже среднего уровня задолженности. В то время как, клиенты, приследующие цель - "автомобиль", чаще остальных задерживают выплаты кредита в срок. На них приходится 9,3%.

### Общий вывод

Отвечая на главный вопрос влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок, можно ответить, что клиенты, находящиеся в статусе не женат/не замужем более других подвержены невыполнению кредитных обязательств в срок. Более надежной категорией по данному показателю следует считать вдовцов и вдов. Исходя из количества детей - идельаными клиентами является те, у кого 5 детей, т.к. по ним не выявлено ни одного случая прострочки. Если брать во внимание невыплату кредита в срок - то наименьшие показатели принадлежат категории людей без детей. Подводя итог можно сказать, что одинокие люди более ответственно относятся к своим финансовым обязательствам.